# CHAPTER 5 범주형 데이터 다루기

* nomianl 범주 : 순서가 없음   
    ex) 파랑, 빨강 , 초록
* ordial 범주 : 순서 있음
    ex) 낮음, 중간, 높음
    
범주형 데이터는 종종 벡터나 문자열로 표현, 그러나 머신러닝에는 수치값을 입력해야하므로 문제가 됨(ex) k-근접 알고리즘)



## 5.1 Nominal Categoricla 특성 Encoding하기

### **one-hot encoding**
* 원본 특성에 있는 클래스마다 이진 특성을 하나씩 생성
* 단어 집합의 크기를 벡터의 차원으로 하고, 표현하고 싶은 단어의 인덱스에 1의 값을 부여하고, 다른 인덱스에는 0을 부여하는 단어의 벡터 표현 방식
* 선형 의존성을 피하기 위해 결과 행렬에서 one-hot encoding된 feature 하나를 삭제하는 것이 좋음
    - EX) 성별을 두 변수로 인코딩하면 is_male과 is_female음의 상관 관계를 갖는 두 가지 기능이 생성되므로 두 기능 중 하나만 사용하여 남성을 말하는 기준을 효과적으로 설정하고 예측 알고리즘에서 is_female 열이 중요한지 확인하는 것이 좋습니다
    
* **LabelBinarizer** method로 객체 생성

   #### LabelEcoder
    * 문자열 타깃 데이터를 정수 rable로 변환할때 사용
    * 두 클래스는 일차원 배열

In [54]:
import numpy as np
from sklearn.preprocessing import LabelBinarizer, MultiLabelBinarizer

feature = np.array([
    ["Texas"],
    ["California"],
    ["Texas"],
    ["Delaware"],
    ["Texas"]
])

one_hot = LabelBinarizer()

one_hot.fit_transform(feature)

array([[0, 0, 1],
       [1, 0, 0],
       [0, 0, 1],
       [0, 1, 0],
       [0, 0, 1]])

* classes_로 클래스 확인

In [55]:
one_hot.classes_

array(['California', 'Delaware', 'Texas'], dtype='<U10')

* one-hot encoding을 되돌리려면 inverse_transform 사용

In [56]:
one_hot.inverse_transform(one_hot.transform(feature))

array(['Texas', 'California', 'Texas', 'Delaware', 'Texas'], dtype='<U10')

* pandas get_dummies를 활용한 one-hot encoding

In [23]:
import pandas as pd

pd.get_dummies(feature[:, 0])

,California,Delaware,Texas
0,0,0,1
1,1,0,0
2,0,0,1
3,0,1,0
4,0,0,1


* **MultiLabelBinarizer** method로 객체 생성

In [24]:
multiclass_feature = [
    ("Texas", "Florida"),
    ("California", "Alabama"),
    ("Texas", "Florida"),
    ("Delaware", "Florida"),
    ("Texas", "Alabama")
]

one_hot_multiclass = MultiLabelBinarizer()

one_hot_multiclass.fit_transform(multiclass_feature)

array([[0, 0, 0, 1, 1],
       [1, 1, 0, 0, 0],
       [0, 0, 0, 1, 1],
       [0, 0, 1, 1, 0],
       [1, 0, 0, 0, 1]])

In [25]:
one_hot_multiclass.classes_

array(['Alabama', 'California', 'Delaware', 'Florida', 'Texas'],
      dtype=object)

* OneHotEncoder 클래스는 기본적으로 희소배열 반환
* spase=False로 지정하면 밀집배열 생성
* 정수도 문자열처럼 취급해 변환

In [61]:
from sklearn.preprocessing import OneHotEncoder

feature= np.array([["텍사스",1],
                   ["캘리포니아",1],
                   ["텍사스",3],
                   ["서울",1],
                   ["텍사스",1]])

onehot=OneHotEncoder(sparse=False)
onehot.fit_transform(feature)

array([[0., 0., 1., 1., 0.],
       [0., 1., 0., 1., 0.],
       [0., 0., 1., 0., 1.],
       [1., 0., 0., 1., 0.],
       [0., 0., 1., 1., 0.]])

* categories_ 속성으로 클래스 확인
* 특정 열에 적용하려면 ColumnTransformer와 함께 사용

In [5]:
onehot.categories_

[array(['서울', '캘리포니아', '텍사스'], dtype='<U5'), array(['1', '3'], dtype='<U5')]

***

## 5.2 ordial categorical 특성 인코딩하기

### pandas df의 **replace** method
* 문자열 ravel을 수치값으로 변환
* class rable 문자열을 정수로 mapping하는 dictionary를 만들고 이를 feature에 적용

In [62]:
import pandas as pd


df = pd.DataFrame({"Score": ["Low", "Low", "Medium", "Medium", "High"]})

# mapping dictionary 생성
scale_mapper = {
    "Low": 1,
    "Medium": 2,
    "High": 3
}

# 특성을 정수로 변환
df_mapper=df["Score"].replace(scale_mapper)


In [10]:
df_mapper

0    1
1    1
2    2
3    2
4    3
Name: Score, dtype: int64

In [12]:
df_mapper.dtype

dtype('int64')

In [14]:
df.shape

(5, 1)

### class간 간격이 동일하지 않을 때

In [20]:
df = pd.DataFrame({"Score": ["Low",
                             "Low",
                             "Medium",
                             "Medium",
                             "High",
                             "Baerly More Than Medium"]})

# mapping dictionary 생성
scale_mapper = {
    "Low": 1,
    "Medium": 2,
    "High": 4,
    "Baerly More Than Medium":3
}

df["Score"].replace(scale_mapper)

0    1
1    1
2    2
3    2
4    4
5    3
Name: Score, dtype: int64

* class에 mapping하는 수치값에 주의하기

In [21]:
scale_mapper = {
    "Low": 1,
    "Medium": 2,
    "High": 4,
    "Baerly More Than Medium":2.1
}

df["Score"].replace(scale_mapper)

0    1.0
1    1.0
2    2.0
3    2.0
4    4.0
5    2.1
Name: Score, dtype: float64

#### **OrdinalEncoder**

* calss범주를 순서대로 변환
* 정수 데이터도 범주형으로 인식해 변환
* 특정 열만 범주형으로 변환하려면 ColumnTransformer와 함께 사용


In [29]:
from sklearn.preprocessing import OrdinalEncoder

features= np.array([["Low",10],
                    ["High",50],
                    ["Meidum",3]
                   ])

ordinal_encoder = OrdinalEncoder()
ordinal_encoder.fit_transform(features)

array([[1., 0.],
       [0., 2.],
       [2., 1.]])

In [28]:
ordinal_encoder.categories_

[array(['High', 'Low', 'Meidum'], dtype='<U6'),
 array(['10', '3', '50'], dtype='<U6')]

***

## 5.3 특성 dictionary 인코딩하기

### **DictVectorizer**
* 0이 아닌 값의 원소만 저장하는 희소 행렬을 반환
* sparse=Fales로 지정하면 밀집 vector출력

In [38]:

from sklearn.feature_extraction import DictVectorizer

#dictionary 생성
data_dict = [
    {"Red": 2, "Blue": 4},
    {"Red": 4, "Blue": 3},
    {"Red": 1, "Yellow": 2},
    {"Red": 2, "Yellow": 2}
]

dictvectorizer = DictVectorizer(sparse=False)

# 딕셔너리를 특성행렬로 변환
features = dictvectorizer.fit_transform(data_dict)

features

array([[4., 2., 0.],
       [3., 4., 0.],
       [0., 1., 2.],
       [0., 2., 2.]])

* get_feature_names 를 사용해 생성된 특성의 이름을 얻을 수 있음

In [10]:
# get feature names
dictvectorizer.get_feature_names()

['Blue', 'Red', 'Yellow']

In [34]:
import pandas as pd

feature_names=dictvectorizer.get_feature_names()
pd.DataFrame(features, columns=feature_names)

,Blue,Red,Yellow
0,4.0,2.0,0.0
1,3.0,4.0,0.0
2,0.0,1.0,2.0
3,0.0,2.0,2.0


* dictvectorizer을 이용해 각 문서에 등장한 단어 횟수를 특성으로 하는 특성행렬을 만들 수 있다

In [39]:
# 네개의 문서에 대한 단어 카운트 dictionary를 생성
doc1={"Red": 2, "Blue": 4}
doc2={"Red": 4, "Blue": 3}
doc3={"Red": 1, "Yellow": 2}
doc4={"Red": 2, "Yellow": 2}

#list 생성
doc_counts=[doc1,doc2,doc3,doc4]

#단어 카운트 dictionary를 특성 행렬로 반환
dictvectorizer.fit_transform(doc_counts)

array([[4., 2., 0.],
       [3., 4., 0.],
       [0., 1., 2.],
       [0., 2., 2.]])

***

## 5.4 누락된 class값 대체하기

### 1. k-최근접 이웃(K-Nearest Neighbot)
* 분류는 각 점의 가장 가까운 이웃들의 단순한 다수결로 계산
* 쿼리 지점은 해당 지점의 가장 가까운 이웃들 내에서 가장 많은 대표자를 가진 데이터 클래스를 할당
* sklearn의 KNeighborsClassifier은 사용자가 지정한 정수 값인 쿼리 포인트의 가장 가까운 이웃을 기반으로 학습 구현
    - weight매개변수: distance - 쿼리포인트에서 거리의 역에 비례하는 가중치 부여
    - weight매개변수 :uniform - 각 이웃에 동일한 가중치 할당

In [41]:
# load libraries
import numpy as np
from sklearn.neighbors import KNeighborsClassifier

X = np.array([[0, 2.10, 1.45],
            [1, 1.18, 1.33],
            [0, 1.22, 1.27],
            [1, -0.21, -1.19]])

X_with_nan = np.array([[np.nan, 0.87, 1.31],
                      [np.nan, -0.67, -0.22]])

# KNN 훈련
clf = KNeighborsClassifier(3, weights='distance')
trained_model = clf.fit(X[:,1:], X[:, 0])

# 누락된 값의 클래스를 예측한다
imputed_values = trained_model.predict(X_with_nan[:, 1:])

# 예측된 클래스와 원본 특성을 열로 합침
X_with_imputed = np.hstack((imputed_values.reshape(-1, 1), X_with_nan[:, 1:]))
# .hstack 배열 순서를 가로로 쌓음

# 두 측성 행렬 연결
np.vstack((X_with_imputed, X))

array([[ 0.  ,  0.87,  1.31],
       [ 1.  , -0.67, -0.22],
       [ 0.  ,  2.1 ,  1.45],
       [ 1.  ,  1.18,  1.33],
       [ 0.  ,  1.22,  1.27],
       [ 1.  , -0.21, -1.19]])

In [43]:
imputed_values

array([0., 1.])

In [44]:
imputed_values.reshape(-1,1)

array([[0.],
       [1.]])

In [42]:
X[:,0]

array([0., 1., 0., 1.])

### 2. 가장 자주 등장하는 값으로 채우기

#### Imputer

* 누락값 채움
* strategy매개변수
    - "mean": 평균값 대입
    - "median": 중간값 대입
    - "most_frequent": 자주등장하는 값으로 대입

In [12]:

from sklearn.preprocessing import Imputer

# 두 행렬을 합침(vstack : 수직으로 배열 )
X_complete = np.vstack((X_with_nan, X))

imputer = Imputer(strategy='most_frequent', axis=0)

imputer.fit_transform(X_complete)

C:\Users\Administrator\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:66: DeprecationWarning: Class Imputer is deprecated; Imputer was deprecated in version 0.20 and will be removed in 0.22. Import impute.SimpleImputer from sklearn instead.
  warnings.warn(msg, category=DeprecationWarning)


array([[ 0.  ,  0.87,  1.31],
       [ 0.  , -0.67, -0.22],
       [ 0.  ,  2.1 ,  1.45],
       [ 1.  ,  1.18,  1.33],
       [ 0.  ,  1.22,  1.27],
       [ 1.  , -0.21, -1.19]])

***

## 5.5 불균형한 class 다루기

많은 데이터를 모으기 위해 
1. 모델에 내장된 가중치 매개변수를 사용 
2. downsampling, upsampling을 고려

### iris dataset background
* 세 개의 클래스(Iris setosa, Iris virginica, Iris versicolor)의 sample 50개씩
* vriginicam versicolor 합침
* setosa sample 40개 삭제, 따라서 setosa sample 10개 그게 아닌 sample 90 -> 불균형한 데이터

In [71]:
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import load_iris

iris = load_iris()

features = iris.data
target = iris.target

# 첫 sample 40개 삭제
features = features[40:, :]
target = target[40:]


# setosa class를 음성 클래스로 하는 이진벡터 생성
target = np.where((target == 0), 0, 1)

target

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

* 불균형한 영향을 줄이기 위해 클래스에 가중치를 부여할 수 있는 매개변수 제공

In [72]:
#가중치 생성
weights = {0: .9, 1: 0.1}

RandomForestClassifier(class_weight=weights)

RandomForestClassifier(bootstrap=True, class_weight={0: 0.9, 1: 0.1},
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, min_impurity_decrease=0.0,
                       min_impurity_split=None, min_samples_leaf=1,
                       min_samples_split=2, min_weight_fraction_leaf=0.0,
                       n_estimators='warn', n_jobs=None, oob_score=False,
                       random_state=None, verbose=0, warm_start=False)

* 가중치를 balanced로 지정해 클래스 빈도에 반비례하게 자동으로 가중치 생성

In [73]:
RandomForestClassifier(class_weight="balanced")

RandomForestClassifier(bootstrap=True, class_weight='balanced',
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, min_impurity_decrease=0.0,
                       min_impurity_split=None, min_samples_leaf=1,
                       min_samples_split=2, min_weight_fraction_leaf=0.0,
                       n_estimators='warn', n_jobs=None, oob_score=False,
                       random_state=None, verbose=0, warm_start=False)

### downsampling
* 다수 클래스의 샘플을 줄임
* 다수 클래스에서 중복을 허용하지 않고 랜덤하게 샘플을 선택해 소수 클래스와 같은 크기의 샘플 부분집합을 만든다



In [74]:
#각 클래스의 sample index 추출
i_class0 = np.where(target == 0)[0]
i_class1 = np.where(target == 1)[0]

# 각 클래스의 sample 개수
n_class0 = len(i_class0)
n_class1 = len(i_class1)

#클래스 0의 sample만큼 클래스 1에서 중복을 허용하지 않고 핸덤하게 샘플을 뽑음
i_class1_downsampled = np.random.choice(i_class1, size=n_class0, replace=False)

#클래스 0의 타깃벡터와 다운샘플링된 클래스 1의 타깃벡터 합침
np.hstack((target[i_class0], target[i_class1_downsampled]))

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

In [75]:
# 클래스 0의 특성 행렬과 다운샘플링된 클래스 1의 특성행렬을 합침
np.vstack((features[i_class0,:], features[i_class1_downsampled, :]))[0:5]

array([[5. , 3.5, 1.3, 0.3],
       [4.5, 2.3, 1.3, 0.3],
       [4.4, 3.2, 1.3, 0.2],
       [5. , 3.5, 1.6, 0.6],
       [5.1, 3.8, 1.9, 0.4]])

### upsampling
* 소수 클래스의 샘플을 늘림
* 다수 클래스의 샘플만큼 소수 클래스에서 중복을 허용해 랜덤하게 샘플 선택
* 결과적으로 다수 클래스와 소수 클래스의 샘플 수가 같아짐

In [51]:
# 클래스 1의 샘플개수만큼 클래스 0에서 중복을 허용해 랜덤하게 샘플 선택
i_class0_upsampled = np.random.choice(i_class0, size=n_class1, replace=True)

#클래스 0의 업샘플링된 타깃벡터와 클래스 1의 타깃벡터를 합침
np.concatenate((target[i_class0_upsampled], target[i_class1]))

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1])

In [52]:
#클래스 0의 업샘플링된 특ㄱ성행렬과 클래스 1의 특성 행렬을 합침
np.vstack((features[i_class0_upsampled,:], features[i_class1,:]))[0:5]

array([[4.6, 3.2, 1.4, 0.2],
       [4.6, 3.2, 1.4, 0.2],
       [5.3, 3.7, 1.5, 0.2],
       [4.5, 2.3, 1.3, 0.3],
       [5. , 3.5, 1.6, 0.6]])